In [1]:
import simpy
import random
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import itertools
import openpyxl
import os
import math
from pathlib import Path
from statistics import NormalDist,stdev,mean

In [2]:
Street_size=[14, 2, 15, 4, 3, 22]
num_buildings= sum(Street_size)
max_capacity=5
num_families= (num_buildings *max_capacity)
class Conditions:
    def __init__(self,name,damages):
        self.name=name
        self.damages=damages

#Create conditions and add them to a global list, a seperate excel sheet is prepared for each condition    
Hatay=Conditions('Hatay',[0.2365,0.2805,0.5475,1])
Adıyaman=Conditions('Adıyaman',[0.100,0.138,0.694,1])
Maraş=Conditions('Maraş',[0.216,0.234,0.576,1])
Antakya=Conditions('Antakya',[0.412,0.487,0.5475,1])
all_conditions=[Hatay,Adıyaman,Maraş,Antakya]

In [3]:
def confidence_interval(data):
    if len(data)<=2:
        return mean(data),0
    confidence=0.90
    dist = NormalDist.from_samples(data)
    z = NormalDist().inv_cdf((1 + confidence) / 2.)
    h = dist.stdev * z / ((len(data) - 1) ** .5)
    return dist.mean,h

In [4]:
def output_graph(Regroup_run_unaccounteds,group_sizes,regroup_times,filepath,condition,yaxis):  
    colors = itertools.cycle(["r", "b", "g", "c", "m", "y", "k","w"])
    finalpath=("{}\\{}.png".format(filepath,condition))
    Regroup_means=[]
    Regroup_Confidences=[]
    fig, ax = plt.subplots()
    for group_runs in Regroup_run_unaccounteds:
        means=[]
        Confidences=[]
        for runs in group_runs:
            mean_unaccounted,CI_unaccounted=confidence_interval(runs)
            means.append(mean_unaccounted)
            Confidences.append(CI_unaccounted)
            
        Regroup_means.append(means)
        Regroup_Confidences.append(Confidences)
        
    for means, Confidences,regroup_time in zip(Regroup_means, Regroup_Confidences,regroup_times):
        current_color=next(colors)
        ax.plot(group_sizes,means,label='RegroupTime={}'.format(regroup_time))
        Lower_CI=[x - y for x,y in zip(means, Confidences)]
        Upper_CI=[x + y for x,y in zip(means, Confidences)]
        ax.fill_between(group_sizes, Lower_CI, Upper_CI, color=current_color, alpha=.1)
        ax.legend()
        ax.set_xlabel("Group Sizes")
        ax.set_ylabel(yaxis)
        ax.set_title(condition)
    fig.savefig(finalpath)

In [5]:
#Adds performance measures to the data frame
def add_sim_data(df,regroup_time,group_size,simtime,run_unaccounted,num_stranded,run_stdevs):
    avg_victim=sum(df.loc["Under Rubble"])/len(df.loc["Under Rubble"])
    avg_not_regrouped=sum(df.loc["Escaped but not regrouped"])/len(df.loc["Escaped but not regrouped"])
    avg_Wrongfully_assigned=sum(df.loc["Wrongfully_assigned"])/len(df.loc["Wrongfully_assigned"])
    avg_Travel_Time_Per_Person=sum(df.loc["Travel_Time"])/len(df.loc["Travel_Time"])
    avg_LentMembers=sum(df.loc["LentMembers"])/len(df.loc["LentMembers"])
    mean_unaccounted,CI_unaccounted=confidence_interval(run_unaccounted)
    LowerCI_run_unaccounted=mean_unaccounted-CI_unaccounted
    UpperCI_run_unaccounted=mean_unaccounted+CI_unaccounted
    avg_stranded=sum(num_stranded)/len(num_stranded)
    avg_stdevs,CI_stdevs=confidence_interval(run_stdevs)
    LowerCI_stdevs=avg_stdevs-CI_stdevs
    UpperCI_stdevs=avg_stdevs+CI_stdevs
    sim_data=pd.DataFrame([LowerCI_run_unaccounted,mean_unaccounted,UpperCI_run_unaccounted,avg_stranded,regroup_time,group_size,simtime,avg_victim,avg_not_regrouped,avg_Wrongfully_assigned,avg_Travel_Time_Per_Person,avg_LentMembers,avg_stdevs,UpperCI_stdevs,LowerCI_stdevs],index=['LowerCI_unaccounted','Avg_unaccounted','UpperCI_unaccounted','Avg_Stranded_Victims','Regroup Time','Group Size','Simulation Time','Average_Under_Rubble','Average_Not_Regrouped','Average Wrongfully Assigned','avg_Travel_Time_Per_Person','avg_LentMembers','avg_stdevs','UpperCI_stdevs','LowerCI_stdevs'])
    new_df = pd.concat([df,sim_data])
    return new_df

In [6]:
#Outputs the data frame to excel, adjusts column widths etc.
def output_to_excel(dfs,filename):
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    for sheetname, df in dfs.items():  # loop through `dict` of dataframes
        df.to_excel(writer, sheet_name=sheetname)  # send df to writer
        worksheet = writer.sheets[sheetname]  # pull worksheet object
        for idx, col in enumerate(df):  # loop through all columns
            series = df[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space
            worksheet.set_column(idx, idx, max_len+5)  # set column width
    writer.close()

In [7]:
class GraphData(object):
    def __init__(self,condition,regroup_time,group_sizes):
        self.condition=condition
        self.regroup_time=regroup_time
        self.group_sizes=group_sizes
        

In [8]:
#Building class, shows no behaviour or process
class Building(object):
    def __init__(self, env, name, capacity, distance_to_regroup,street):
        self.seed= random.random()
        self.env = env
        self.name = name
        self.capacity = capacity
        self.families = []
        self.status=3
        self.distance=distance_to_regroup
        self.groups=[]
        global selected_conditions
        for damage in selected_conditions.damages:
            if self.seed< damage:
                self.status=selected_conditions.damages.index(damage)
                break
        self.street= street
    def add_family(self, env, family):
        if len(self.families)<self.capacity:
            self.families.append(family)
            family.location=self
            family.residency=self
            family.floor=self.families.index(family)+1
            return 1
        else:
            return 0

In [9]:
#Family class, "behaviour" method is a simpy env process dictating its evolution
class Family(object):
    def __init__(self, env, name):
        self.env = env
        self.name = name
        self.responsibility = None 
        self.group= None
        self.free=True
        self.location=None
        self.residency=None
        self.occupied=False
        self.travel_time=None
        self.floor=None
        self.messenger_for=None
        self.messenger=None
        self.rescuer=None
        self.response_time=None
        self.hasaccountson=[self]
        self.disorderly=random.randrange(0,5)
    def behavior(self, env, regroup_location,regroup_time,dayjob,under_rubble,wrongfully_assigned,travel_times,time):
        #env.now() can be called to retrieve environment(simulation) time
        #yielding events are cumulative, self.env.timeout(1) puts the object on hold for one tick and then proceeds to the next line
        yield self.env.timeout(0)
        if self.residency.status== 0 or  self.residency.status== 1:
            if self.floor!=1:
                self.free=False
                under_rubble.append(self)
            if self.floor==1 and random.random()<=0.8:
                self.free=False
                under_rubble.append(self)
        if time=='Day':
            if random.random()<=0.4:
                self.free=True
                self.location=dayjob
                if self in under_rubble:
                    under_rubble.remove(self)
                if random.random()<=0.25:
                    self.free=False
                    if self not in under_rubble:
                        under_rubble.append(self)
                 

        if self.free:
            #Defines the escape method as a simpy process
            env.process(self.escape(env,regroup_location,travel_times))    
            #env.now = 0+regroup_time
            yield self.env.timeout(regroup_time)
            
        yield self.env.timeout(2)
        
        if self.free==False:
            while(1):
                if self.rescuer!=None and self.messenger!=None:
                    if self.rescuer.location==self.residency:
                        self.response_time=self.env.now
                        break
                yield self.env.timeout(1)        
        #env.now = 0+regroup_time+2
        #Moves to responsibility location if assigned to a victim
        if self.free:
            #Rescuer check
            while(1):
                if self.responsibility is not None and self.location==regroup_location:
                    if self.responsibility.group != self.group:
                        yield self.env.timeout(round(self.responsibility.group.size/5))
                    self.travel_time=round(random.expovariate(1/self.responsibility.residency.distance))
                    yield self.env.timeout(self.travel_time)
                    self.location=self.responsibility.residency
                    travel_times.append(self.travel_time)
                    if self.responsibility.residency.status==2 or self.responsibility.residency.status==3:
                        self.responsibility.group.dontrequesthelpforthismember.append(self.responsibility)
                        self.responsibility=None
                        if self not in wrongfully_assigned:
                             wrongfully_assigned.append(self)
                    while(1):
                        yield self.env.timeout(1)
                        if self.responsibility is None and self.location!=regroup_location:
                            self.travel_time=round(random.expovariate(1/self.location.distance))
                            travel_times.append(self.travel_time)
                            yield self.env.timeout(self.travel_time)
                            self.location=regroup_location
                            travel_times.append(self.travel_time)
                            break
                #Rescuer check end
                #Messenger behavior
                if self.messenger_for is not None:
                    if self.location!=regroup_location:
                        self.travel_time=round(random.expovariate(1/self.location.distance))
                        yield self.env.timeout(self.travel_time)
                        self.location=regroup_location
                    waiting_time=0
                    while(1):
                        yield self.env.timeout(1)
                        waiting_time+=1
                        if self.messenger_for.location==regroup_location and self.messenger_for not in self.hasaccountson:
                            self.hasaccountson.append(self.messenger_for)
                            break
                        if waiting_time>=2 *self.messenger_for.residency.distance:
                            break        
                    #Messenger moves to and from victims house
                    while(1):
                        yield self.env.timeout(1)
                        if self.location==regroup_location:
                            if self.messenger_for.location==regroup_location and self.messenger_for not in self.hasaccountson:
                                self.messenger_for.group.dontrequesthelpforthismember.append(self.messenger_for)
                                self.hasaccountson.append(self.messenger_for)
                            self.travel_time=round(random.expovariate(1/self.messenger_for.residency.distance))
                            yield self.env.timeout(self.travel_time)
                            travel_times.append(self.travel_time)
                            self.location=self.messenger_for.residency
                        if self.messenger_for.rescuer.location==self.messenger_for.residency and self.messenger_for in self.hasaccountson:
                            self.messenger_for.messenger=None                        
                            self.messenger_for.rescuer.responsibility=None
                            self.messenger_for.rescuer=None
                            self.messenger_for=None
                            yield self.env.timeout(self.travel_time)
                            self.location=regroup_location
                            break
                            
                        if self.location!=regroup_location:    
                            self.travel_time=round(random.expovariate(1/self.location.distance))
                            yield self.env.timeout(self.travel_time)
                            self.location=regroup_location
                            travel_times.append(self.travel_time)
           #Messenger behavior end

                yield self.env.timeout(1)
    def escape(self,env,regroup_location,travel_times):
        if self.free:
        #if building that this family resides in is status 3 or 2, family escapes, otherwise not
            self.travel_time= round(random.expovariate(1/self.location.distance))+round(random.expovariate(1/(self.floor)))
            self.travel_time+=self.disorderly   
            travel_times.append(self.travel_time)
            yield self.env.timeout(self.travel_time)
            self.location=regroup_location
            regroup_location.families.append(self)
    def assign_victim(self,env,victim):
        #assigns this family a responsibility(family)
        if self.occupied or victim.rescuer is not None:
            return 0
        else:
            self.responsibility=victim
            self.occupied=True
            victim.rescuer=self
            return 1

In [10]:
#Group object, holds a list of families,and has a behaviour method as a simpy process 
class Group(object):
    def __init__(self, env, name,group_size):
        self.env = env
        self.name=name
        self.size=group_size
        self.families=[]
        self.accesible=[]
        self.available=[]
        self.checked=[]
        self.unaccounted_for=[]
        self.nomessengervictim=[]
        self.regroup_location=None
        self.needs_help=False
        self.dontrequest=False
        self.helping_group=None
        self.early_regroup=True
        self.getting_help_from=None
        self.lentmember=[]
        self.dontrequesthelpforthismember=[]
    def behavior(self, env,regroup_time,regroup_location,escaped_but_not_regrouped):
        self.regroup_location=regroup_location
        self.regroup_location.groups.append(self)
        
        while(1):
            self.early_regroup=True
            if env.now>=regroup_time:
                break
            for family in self.families:
                if family.location!=regroup_location:
                    self.early_regroup=False
                    
                    
            if self.early_regroup==True:
                yield self.env.timeout(self.regroup(env,regroup_location))
                self.coordinate(env)
                break
            yield self.env.timeout(1)
            
            
            
            
        yield self.env.timeout(1)
        for family in self.families:
            if family.free and family.location!=regroup_location:
                escaped_but_not_regrouped.append(family)
        yield self.env.timeout(self.regroup(env,regroup_location))
        self.coordinate(env)        
        while(1):
            yield self.env.timeout(self.regroup(env,regroup_location))
            self.coordinate(env)
            if self.needs_help==False:
                for group_in_need in self.regroup_location.groups:
                    if group_in_need.needs_help==True and group_in_need.getting_help_from==None and group_in_need.dontrequest==False:
                        self.helping_group=group_in_need
                        group_in_need.getting_help_from=self
                        group_in_need.dontrequest=True
                            
                        
                
            if self.helping_group!=None and self.helping_group.getting_help_from==self and self.needs_help==False and self.helping_group.needs_help==True:
                for family in self.families:
                    if family.responsibility==None and family.messenger_for==None and family in self.available and family.location==regroup_location:
                        self.helping_group.available.append(family)
                        self.available.remove(family)
                        self.helping_group.coordinate(env)
                        if family not in self.lentmember:
                            self.lentmember.append(family)
                            #Member Lending Penalty Time
                            # *************
                            # *************
                            #Change the number below
                            yield self.env.timeout(1)
                        break          
            if self.helping_group is not None and self.helping_group.needs_help==False:
                self.helping_group.getting_help_from=None
                self.helping_group.dontrequest=True
                self.helping_group=None 
            yield self.env.timeout(1)      
                
    def regroup(self,env,regroup_location):
        Account_Time=1
        for family in self.families:
            if family.location==regroup_location:
                if family in self.unaccounted_for:
                    self.unaccounted_for.remove(family)
                if family not in self.available:
                    self.available.append(family)
                if family not in self.accesible:
                    self.accesible.append(family)

        for family1 in self.families:
            for family2 in self.families:
                if family1.location==family2.location and family1.rescuer==family2 and family1 not in family2.hasaccountson:
                    family2.hasaccountson.append(family1)
                if family2.location==family1.location and family1.free and family2.free:
                    #Account_Time+=((len(family1.hasaccountson)+len(family2.hasaccountson))/100)
                    for family in family1.hasaccountson:
                        if family not in family2.hasaccountson:
                            #Account_Time+=(1/20)
                            family2.hasaccountson.append(family)
                    for family in family2.hasaccountson:
                        if family not in family1.hasaccountson:
                            #Account_Time+=(1/20)
                            family1.hasaccountson.append(family)        
                    if family1 in self.unaccounted_for and family1 not in self.available:
                        self.accesible.append(family1)
                        self.available.append(family1)
                        self.unaccounted_for.remove(family1)
                    if family2 in self.unaccounted_for and family2 not in self.available:
                        self.accesible.append(family2)
                        self.available.append(family2)
                        self.unaccounted_for.remove(family2)
                    for family1account in family1.hasaccountson:
                        for family2account in family2.hasaccountson:
                            if family1account==family2account and family1account in self.unaccounted_for:
                                self.unaccounted_for.remove(family1account)
        return math.ceil(Account_Time)                
        #Appends regrouped members to available list, removes them from the unaccountedfor list 
    def coordinate(self, env):
        #Assigns available group members to victims
        for rescuer in self.available:
            for victim in self.unaccounted_for:
                if victim in self.dontrequesthelpforthismember:
                    continue
                if rescuer.assign_victim(env,victim):
                    self.nomessengervictim.append(victim)
                    self.available.remove(rescuer)
                    break
        for messenger in self.available:
            for victim in self.nomessengervictim:
                if messenger.occupied==False and victim.messenger==None:
                    messenger.messenger_for=victim
                    victim.messenger=messenger
                    messenger.occupied=True
                    self.nomessengervictim.remove(victim)
                    self.available.remove(messenger)
                    break
   
        
        for victim in self.unaccounted_for:
            if victim in self.dontrequesthelpforthismember:
                continue
            if victim.rescuer is None or victim.messenger is None:
                # *************
                # *************
                # *************
                #comment out the line below for no assistance between groups
                #self.needs_help=True
                break
            else:
                self.needs_help=False

        
    def add_family(self, env, family):
        if family in self.families:
            return 0
        #Assigns families to the group until it is full
        if len(self.families)<self.size:
            self.families.append(family)
            self.unaccounted_for.append(family)
            family.group=self
            return 1
        else:
            return 0               

In [11]:
#Street class,holds buildings     
class Street(object):
    def __init__(self, env, name, size):
        self.env=env
        self.name=name
        self.start=None
        self.end=None
        self.buildings=[]
        self.size=size
    def add_building(self,env, building):
        if len(self.buildings)<self.size:
            self.buildings.append(building)
            return 1
        else:
            return 0     

In [12]:
def setup(env,num_groups,group_size,families,groups,buildings,streets):
    #Creates buildings streets,families,groups
    streets.extend([Street(env, "6.2 sk.",4), Street(env, "Cami.2 sk.",3), Street(env, "6.1 sk.",15),Street(env, "Aslı Çk.",14), Street(env, "Cami.1 sk.",22), Street(env, "Hisarüstü sk.",2)])
    #intersections.extend([Intersection(env) for i in range(num_intersections)])
    groups.extend([Group(env, 'Group {}'.format(i+1),group_size) for i in range(num_groups)])
    for k in range(len(streets)):
        if k==0:
            distance_of_street=0
        elif k>=1 and k<=2:
            distance_of_street=streets[0].size
        elif k>=3 and k<=4:
            distance_of_street=streets[0].size+streets[1].size
        elif k==5:
            distance_of_street=streets[0].size+streets[2].size
        for i in range(streets[k].size):
            new_building=Building(env, 'Building {}.{}'.format(i+1,streets[k].name), max_capacity, ((i+1)+distance_of_street),streets[k])
            buildings.append(new_building)
            
    families.extend([Family(env, 'Family {}'.format(i+1)) for i in range(num_families)]) 
    for family in families:
        for building in buildings:
            if building.add_family(env,family):
                break

    for building in buildings:
        for street in streets:
            if street.add_building(env,building): 
                break    

In [13]:
#Controls the distribution of group members
def assign_groups(env, families, groups,Restrict_assignment):
    if Restrict_assignment==False:
        for family in temp_fam:
            if family.group==None:
                for group in groups:
                    if group.add_family(env,family):
                        break
    if Restrict_assignment==True:
        temp_fam=families
        random.shuffle(temp_fam)
        restrict=False
        for family in temp_fam:
            if family.group!=None:
                continue
            for group in groups:
                restrict=False
                if len(group.families)>0:
                    for already_member in group.families:
                        if family.residency==already_member.residency and family!=already_member:
                            restrict=True
                            break
                if restrict==False:            
                    if group.add_family(env,family):
                        break


        for family in temp_fam:
            if family.group==None:
                for group in groups:
                    if group.add_family(env,family):
                        break

In [14]:
def run_sim(env,sim_time,regroup_time,families,groups,under_rubble,escaped_but_not_regrouped,wrongfully_assigned,travel_times,time):
    #Defines simulation processes and completes a single run
    regroup_location=Building(env, 'Regroup Location', 999, 0, None)
    DayJob=Building(env, 'DayJob', 999, 120, None) 
    for family in families:
        env.process(family.behavior(env, regroup_location,regroup_time,DayJob,under_rubble,wrongfully_assigned,travel_times,time))
    for group in groups:
        env.process(group.behavior(env,regroup_time,regroup_location,escaped_but_not_regrouped))
    env.run(until=sim_time)

In [15]:
def batch_run(sim_time,regroup_time,group_size,time,batch_size):
    #Runs multiple instances of the simulation and averages out performance measures
    num_under_rubble=[]
    num_escaped_but_not_regrouped=[]
    num_wrongfully_assigned=[]
    average_travel_times=[]
    num_stranded=[]
    num_LentMembers=[]
    batch_unaccounted_in_groups=[]
    run_stdevs=[]
    run_unaccounted=[]
    run_response_times=[]
    average_unaccounted=[]
    for i in range(batch_size):
        wrongfully_assigned=[]
        families = []
        groups=[]
        LentMembers=[]
        buildings=[]
        Response_times=[0]
        streets=[]
        travel_times=[]
        under_rubble=[]
        escaped_but_not_regrouped=[]
        stranded=[]
        free_families=0
        Unaccounted_in_groups=[]
        env = simpy.Environment()
        num_groups= round(num_families/group_size)
        setup(env,num_groups,group_size,families,groups,buildings,streets)
        assign_groups(env, families, groups,False)
        run_sim(env,sim_time,regroup_time,families,groups,under_rubble,escaped_but_not_regrouped,wrongfully_assigned,travel_times,time)
        num_under_rubble.append(len(under_rubble))
        num_escaped_but_not_regrouped.append(len(escaped_but_not_regrouped))
        num_wrongfully_assigned.append(len(wrongfully_assigned))
        for family in families:
            if family.free:
                free_families+=1
        average_travel_times.append(sum(travel_times)/free_families)
        for family in families:
            if family.free==False and family in family.group.unaccounted_for and family.rescuer==None:
                stranded.append(family)
            if family.free==False and family.response_time!=None:
                Response_times.append(family.response_time)
        for group in groups:
            Unaccounted_in_groups.append(len(group.unaccounted_for))
            LentMembers.append(len(group.lentmember))
            
        run_response_times.append(max(Response_times))    
        run_unaccounted.append(sum(Unaccounted_in_groups))
        average_unaccounted.append(mean(Unaccounted_in_groups))
        
        if len(Unaccounted_in_groups)>=2:
            if mean(Unaccounted_in_groups)!=0:
                run_stdevs.append(stdev(Unaccounted_in_groups)/mean(Unaccounted_in_groups))
            else:
                run_stdevs.append(stdev([0,1]))
        else:
            run_stdevs.append(stdev([0,1]))
        num_LentMembers.append(sum(LentMembers))
        num_stranded.append(len(stranded))    
    #Holds calculated performance measures to print
    Lists= (num_under_rubble,num_escaped_but_not_regrouped, num_wrongfully_assigned,average_travel_times,num_stranded,num_LentMembers,run_unaccounted,average_unaccounted,run_stdevs)
    df=pd.DataFrame(Lists,index=['Under Rubble','Escaped but not regrouped','Wrongfully_assigned','Travel_Time','Stranded','LentMembers','Unaccounted_For','Average_Unaccounted_Group','Unacc_stdev_group'])
    Sim_DataFrame=add_sim_data(df,regroup_time,group_size,sim_time,run_unaccounted,num_stranded,run_stdevs)
    return Sim_DataFrame,run_unaccounted,run_stdevs,average_travel_times,run_response_times

In [16]:
#Dictionary of dataframes to be printed to seperate excel sheets
work_dir = os.getcwd() 
work_dir = os.path.join(work_dir, 'Desktop')
times=['Day','Night']
regroup_times=[120,150,180,30,60,90]
group_sizes=[2,4,3,5,6,10,15,20,30,50]
sim_times=[240]
for time in times:
    
    #There is a better way to hold these graphs, but I dont have time, so here you go
    Regroup_Hatay_Graph=[]
    Regroup_Adıyaman_Graph=[]
    Regroup_Maraş_Graph=[]
    Regroup_Antakya_Graph=[]
    Regroup_Hatay_stdev_Graph=[]
    Regroup_Adıyaman_stdev_Graph=[]
    Regroup_Maraş_stdev_Graph=[]
    Regroup_Antakya_stdev_Graph=[]
    Regroup_Hatay_travel_Graph=[]
    Regroup_Adıyaman_travel_Graph=[]
    Regroup_Maraş_travel_Graph=[]
    Regroup_Antakya_travel_Graph=[]
    Regroup_Hatay_response_Graph=[]
    Regroup_Adıyaman_response_Graph=[]
    Regroup_Maraş_response_Graph=[]
    Regroup_Antakya_response_Graph=[]
    
    for regroup_time in regroup_times:
        Hataygraphlist=[]
        Adıyamangraphlist=[]
        Maraşgraphlist=[]
        Antakyagraphlist=[]
        Hataystdevgraphlist=[]
        Adıyamanstdevgraphlist=[]
        Maraşstdevgraphlist=[]
        Antakyastdevgraphlist=[]
        Hatay_travel_graphlist=[]
        Adıyaman_travel_graphlist=[]
        Maraş_travel_graphlist=[]
        Antakya_travel_graphlist=[]
        Hatay_Response_graphlist=[]
        Adıyaman_Response_graphlist=[]
        Maraş_Response_graphlist=[]
        Antakya_Response_graphlist=[]
        for group_size in group_sizes:
            for sim_time in sim_times:
                DataFrames_dict = {"Hatay": pd.DataFrame(), "Adıyaman":pd.DataFrame(), "Maraş":pd.DataFrame(), "Antakya":pd.DataFrame()}
                for conditions in all_conditions:
                    selected_conditions=conditions
                    #Set last parameter to desired batch size
                    # * * * * * * * * * *
                    # ******
                    new_batch,run_unaccounted,run_stdevs,average_travel_times,run_response_times=batch_run(sim_time,regroup_time,group_size,time,20)
                    if selected_conditions.name=='Hatay':
                        Hataygraphlist.append(run_unaccounted)
                        Hataystdevgraphlist.append(run_stdevs)
                        Hatay_travel_graphlist.append(average_travel_times)
                        Hatay_Response_graphlist.append(run_response_times)
                    if selected_conditions.name=='Adıyaman':
                        Adıyamangraphlist.append(run_unaccounted)
                        Adıyamanstdevgraphlist.append(run_stdevs)
                        Adıyaman_travel_graphlist.append(average_travel_times)
                        Adıyaman_Response_graphlist.append(run_response_times)
                    if selected_conditions.name=='Maraş':
                        Maraşgraphlist.append(run_unaccounted)
                        Maraşstdevgraphlist.append(run_stdevs)
                        Maraş_travel_graphlist.append(average_travel_times)
                        Maraş_Response_graphlist.append(run_response_times)
                    if selected_conditions.name=='Antakya':
                        Antakyagraphlist.append(run_unaccounted)
                        Antakyastdevgraphlist.append(run_stdevs)
                        Antakya_travel_graphlist.append(average_travel_times)
                        Antakya_Response_graphlist.append(run_response_times)
                    DataFrames_dict[conditions.name]=pd.concat([DataFrames_dict[conditions.name],new_batch])
                #Change this filepath (name section) for varying approaches
                filepath="{}\\Batches\\Random_Grouping\\SimulationTime{}\\{}\\RegroupTime={}".format(work_dir,sim_time,time,regroup_time)
                Path(filepath).mkdir(parents=True, exist_ok=True)
                finalpath=("{}\\Groupsize{}.xlsx".format(filepath,group_size))
                #Delete top three lines and use the bottom line and change "(name)"to your user name if script fails to run
                #filepath="C:\\Users\\Name\\Desktop\\light_damage_ddDelays_with_disorder.xlsx"
                output_to_excel(DataFrames_dict,finalpath)
                
        Regroup_Hatay_Graph.append(Hataygraphlist)
        Regroup_Adıyaman_Graph.append(Adıyamangraphlist)
        Regroup_Maraş_Graph.append(Maraşgraphlist)
        Regroup_Antakya_Graph.append(Antakyagraphlist)
        
        Regroup_Hatay_stdev_Graph.append(Hataystdevgraphlist)
        Regroup_Adıyaman_stdev_Graph.append(Adıyamanstdevgraphlist)
        Regroup_Maraş_stdev_Graph.append(Maraşstdevgraphlist)
        Regroup_Antakya_stdev_Graph.append(Antakyastdevgraphlist)
        
        Regroup_Hatay_travel_Graph.append(Hatay_travel_graphlist)
        Regroup_Adıyaman_travel_Graph.append(Adıyaman_travel_graphlist)
        Regroup_Maraş_travel_Graph.append(Maraş_travel_graphlist)
        Regroup_Antakya_travel_Graph.append(Antakya_travel_graphlist)
        
        Regroup_Hatay_response_Graph.append(Hatay_Response_graphlist)
        Regroup_Adıyaman_response_Graph.append(Adıyaman_Response_graphlist)
        Regroup_Maraş_response_Graph.append(Maraş_Response_graphlist)
        Regroup_Antakya_response_Graph.append(Antakya_Response_graphlist)
        
    pngfilepath="{}\\Graphs\\Unaccounted People\\{}".format(work_dir,time)
    Path(pngfilepath).mkdir(parents=True, exist_ok=True)
    output_graph(Regroup_Hatay_Graph,group_sizes,regroup_times,pngfilepath,"Hatay","Unaccounted People")
    output_graph(Regroup_Adıyaman_Graph,group_sizes,regroup_times,pngfilepath,"Adıyaman","Unaccounted People")
    output_graph(Regroup_Maraş_Graph,group_sizes,regroup_times,pngfilepath,"Maraş","Unaccounted People")
    output_graph(Regroup_Antakya_Graph,group_sizes,regroup_times,pngfilepath,"Antakya","Unaccounted People")
    
    stdev_pngfilepath="{}\\Graphs\\STDEV_GROUP\\{}".format(work_dir,time)
    Path(stdev_pngfilepath).mkdir(parents=True, exist_ok=True)
    output_graph(Regroup_Hatay_stdev_Graph,group_sizes,regroup_times,stdev_pngfilepath,"Hatay","Relative Group Unaccounted Standard Deviation")
    output_graph(Regroup_Adıyaman_stdev_Graph,group_sizes,regroup_times,stdev_pngfilepath,"Adıyaman","Relative Group Unaccounted Standard Deviation")
    output_graph(Regroup_Maraş_stdev_Graph,group_sizes,regroup_times,stdev_pngfilepath,"Maraş","Relative Group Unaccounted Standard Deviation")
    output_graph(Regroup_Antakya_stdev_Graph,group_sizes,regroup_times,stdev_pngfilepath,"Antakya","Relative Group Unaccounted Standard Deviation")
    
    Travel_pngfilepath="{}\\Graphs\\TravelTimes\\{}".format(work_dir,time)
    Path(Travel_pngfilepath).mkdir(parents=True, exist_ok=True)
    output_graph(Regroup_Hatay_travel_Graph,group_sizes,regroup_times,Travel_pngfilepath,"Hatay","Travel Times")
    output_graph(Regroup_Adıyaman_travel_Graph,group_sizes,regroup_times,Travel_pngfilepath,"Adıyaman","Travel Times")
    output_graph(Regroup_Maraş_travel_Graph,group_sizes,regroup_times,Travel_pngfilepath,"Maraş","Travel Times")
    output_graph(Regroup_Antakya_travel_Graph,group_sizes,regroup_times,Travel_pngfilepath,"Antakya","Travel Times")
    
    Response_pngfilepath="{}\\Graphs\\ResponseTimes\\{}".format(work_dir,time)
    Path(Response_pngfilepath).mkdir(parents=True, exist_ok=True)
    output_graph(Regroup_Hatay_response_Graph,group_sizes,regroup_times, Response_pngfilepath,"Hatay"," Maximum Response Time")
    output_graph(Regroup_Adıyaman_response_Graph,group_sizes,regroup_times, Response_pngfilepath,"Adıyaman"," Maximum Response Time")
    output_graph(Regroup_Maraş_response_Graph,group_sizes,regroup_times, Response_pngfilepath,"Maraş"," Maximum Response Time")
    output_graph(Regroup_Antakya_response_Graph,group_sizes,regroup_times, Response_pngfilepath,"Antakya"," Maximum Response Time")

KeyboardInterrupt: 

In [ ]:
#batch_run(240,80,5,1)